# AgentCore 런타임(Runtime)에서 런타임(Runtime) 컨텍스트 및 세션 관리 이해하기

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)에서 런타임(Runtime) 컨텍스트와 세션 관리를 이해하고 작업하는 방법을 배웁니다. 이 예제는 AgentCore 런타임(Runtime)이 세션을 처리하고, 여러 호출(Invocation)에 걸쳐 컨텍스트를 유지하며, 에이전트(Agent)가 컨텍스트 객체를 통해 런타임(Runtime) 정보에 액세스하는 방법을 보여줍니다.

Amazon Bedrock AgentCore 런타임(Runtime)은 각 사용자 상호작용에 대해 격리된 세션을 제공하여, 에이전트(Agent)가 여러 호출(Invocation)에 걸쳐 컨텍스트와 상태를 유지하면서 서로 다른 사용자 간의 완전한 보안 격리를 보장합니다.

### 튜토리얼 세부사항

|정보| 세부사항|
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 컨텍스트 및 세션 관리|
| 에이전트(Agent) 유형 | 단일         |
| 에이전틱 프레임워크(Agentic Framework)   | Strands Agents |
| LLM 모델           | Anthropic Claude Haiku 4.5 |
| 튜토리얼 구성 요소 | 런타임(Runtime) 컨텍스트, 세션 관리, AgentCore 런타임(Runtime), Strands 에이전트(Agent) 및 Amazon Bedrock 모델 |
| 튜토리얼 분야   | 범용                                                                   |
| 예제 복잡도  | 중급                                                                     |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3|

### 튜토리얼 아키텍처

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)이 세션을 관리하고 에이전트(Agent)에 컨텍스트를 제공하는 방법을 탐색합니다. 다음을 시연합니다:

1. **세션 연속성**: 동일한 세션 ID가 여러 호출(Invocation)에 걸쳐 컨텍스트를 유지하는 방법
2. **컨텍스트 객체**: 에이전트(Agent)가 컨텍스트 매개변수를 통해 런타임(Runtime) 정보에 액세스하는 방법
3. **세션 격리**: 서로 다른 세션 ID가 완전히 격리된 환경을 만드는 방법
4. **페이로드(Payload) 유연성**: 페이로드(Payload)를 통해 에이전트(Agent)에 커스텀 데이터를 전달하는 방법

데모 목적으로 이러한 세션 관리 기능을 보여주는 Strands 에이전트(Agent)를 사용합니다.

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* **세션 기반 컨텍스트 관리**: AgentCore 런타임(Runtime)이 세션 내에서 컨텍스트를 유지하는 방법 이해
* **런타임(Runtime) 세션 수명 주기**: 세션 생성, 유지 및 종료에 대해 학습
* **컨텍스트 객체 액세스**: 컨텍스트 매개변수를 통해 세션 ID 같은 런타임(Runtime) 정보에 액세스
* **세션 격리**: 서로 다른 세션이 완전한 격리를 제공하는 방법 시연
* **페이로드(Payload) 처리**: 커스텀 페이로드(Payload) 구조를 통한 유연한 데이터 전달
* **호출(Invocation) 간 상태**: 동일한 세션 내에서 여러 호출(Invocation)에 걸쳐 에이전트(Agent) 상태 유지

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker 실행 중

## Amazon Bedrock AgentCore 런타임(Runtime) 세션 이해

코드에 들어가기 전에 Amazon Bedrock AgentCore 런타임(Runtime)이 세션을 관리하는 방법을 이해하는 것이 중요합니다:

### 세션 격리 및 보안

AgentCore 런타임(Runtime)은 전용 microVM을 통해 **완전한 세션 격리**를 제공합니다:

- **전용 리소스**: 각 세션은 격리된 CPU, 메모리(Memory), 파일 시스템이 있는 자체 microVM에서 실행
- **보안 경계**: 사용자 세션 간의 완전한 분리로 데이터 오염 방지
- **결정적 정리**: 세션 완료 후 microVM이 종료되고 메모리(Memory)가 소독됨

### 세션 수명 주기

AgentCore 런타임(Runtime)의 세션은 특정 수명 주기를 따릅니다:

1. **생성**: 고유한 `runtimeSessionId`로 첫 번째 호출(Invocation) 시 새 세션 생성
2. **활성 상태**: 세션이 요청을 처리하고 컨텍스트 유지
3. **유휴 상태**: 세션이 컨텍스트를 보존하면서 다음 호출(Invocation) 대기
4. **종료**: 다음으로 인해 세션 종료:
   - 비활성 (15분)
   - 최대 수명 (8시간)
   - 헬스 체크 실패

### 컨텍스트 지속성

세션 내에서 AgentCore 런타임(Runtime)은 다음을 유지합니다:
- **대화 기록**: 이전 상호작용 및 응답
- **애플리케이션 상태**: 실행 중 생성된 변수 및 객체
- **파일 시스템**: 세션 중 생성 또는 수정된 파일
- **환경 변수**: 커스텀 설정 및 구성(Configuration)

### 세션 관리 모범 사례

- **고유 세션 ID**: 각 사용자 또는 대화에 대해 고유한 세션 ID 생성
- **컨텍스트 재사용**: 관련 호출(Invocation)에 동일한 세션 ID를 사용하여 컨텍스트 유지
- **세션 경계**: 다른 사용자 또는 관련 없는 대화에는 다른 세션 ID 사용
- **임시적 특성**: 영구 데이터 저장을 위해 세션에 의존하지 않음 (지속성을 위해 AgentCore 메모리(Memory) 사용)

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agent to AgentCore Runtime to demonstrate session management and context handling. Our agent will showcase how to:

1. **Access Runtime Context**: Use the `context` parameter to get session information
2. **Handle Custom Payloads**: Process structured data passed through the payload
3. **Maintain Session State**: Keep track of user interactions within a session
4. **Demonstrate Session Boundaries**: Show how different sessions are isolated

### Understanding the Context Object

The `context` object in AgentCore Runtime provides valuable information about the current execution environment:

- **session_id**: The current runtime session identifier
- **Runtime Metadata**: Information about the runtime environment
- **Execution Details**: Context about the current invocation

### Strands Agent with Context Handling

Let's look at our implementation that demonstrates session management and context handling:

In [ ]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time.
    Always start by acknowledging the user's name 
    """
)

def get_user_name(user_id):
    users = {
        "1": "Maira",
        "2": "Mani",
        "3": "Mark",
        "4": "Ishan",
        "5": "Dhawal"
    }
    return users[user_id]
    
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    AgentCore Runtime entrypoint that demonstrates context handling and session management.
    
    Args:
        payload: The input payload containing user data and request information
        context: The runtime context object containing session and execution information
    
    Returns:
        str: The agent's response incorporating context information
    """
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # Access runtime context information
    print("=== Runtime Context Information ===")
    print("User id:", user_id)
    print("User Name:", user_name)
    print("User input:", user_input)
    print("Runtime Session ID:", context.session_id)
    print("Context Object Type:", type(context))
    print("=== End Context Information ===")
    
    # Create a personalized prompt that includes context information
    prompt = f"""My name is {user_name}. Here is my request: {user_input}
    
    Additional context: This is session {context.session_id}. 
    Please acknowledge my name and provide assistance."""
    
    response = agent(prompt)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## AgentCore 런타임(Runtime) 배포(Deployment)를 위한 에이전트(Agent) 준비

이제 세션 관리와 컨텍스트 처리를 시연하기 위해 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)해 보겠습니다. 에이전트(Agent)는 다음을 수행하는 방법을 보여줍니다:

1. **런타임(Runtime) 컨텍스트 액세스**: `context` 매개변수를 사용하여 세션 정보 가져오기
2. **커스텀 페이로드(Payload) 처리**: 페이로드(Payload)를 통해 전달된 구조화된 데이터 처리
3. **세션 상태 유지**: 세션 내 사용자 상호작용 추적
4. **세션 경계 시연**: 서로 다른 세션이 어떻게 격리되는지 표시

### 컨텍스트 객체 이해

AgentCore 런타임(Runtime)의 `context` 객체는 현재 실행 환경에 대한 유용한 정보를 제공합니다:

- **session_id**: 현재 런타임(Runtime) 세션 식별자
- **런타임(Runtime) 메타데이터**: 런타임(Runtime) 환경에 대한 정보
- **실행 세부정보**: 현재 호출(Invocation)에 대한 컨텍스트

### 컨텍스트 처리가 있는 Strands 에이전트(Agent)

세션 관리와 컨텍스트 처리를 시연하는 구현을 살펴보겠습니다:

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_context"
)

## AgentCore 런타임(Runtime)에서의 세션 관리 이해

위의 코드는 AgentCore 런타임(Runtime)이 세션을 관리하고 에이전트(Agent)에 컨텍스트를 제공하는 방법에 대한 여러 핵심 개념을 보여줍니다:

### 컨텍스트 객체 구조

진입점 함수의 `context` 매개변수는 런타임(Runtime) 정보에 대한 액세스를 제공합니다:

```python
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    # 세션 정보 액세스
    session_id = context.session_id
    # 에이전트(Agent) 로직에서 컨텍스트 정보 사용
```

### 세션 연속성 이점

단일 세션 내에서 AgentCore 런타임(Runtime)은 다음을 제공합니다:

1. **지속적 환경**: 변수와 상태가 호출(Invocation) 간에 유지됨
2. **컨텍스트 보존**: 에이전트(Agent)가 이전 상호작용을 참조할 수 있음
3. **리소스 재사용**: 초기화된 모델과 도구(Tool)가 로드된 상태로 유지됨
4. **성능 이점**: 후속 호출(Invocation)에 대한 콜드 스타트 시간 단축

### 세션 격리 보장

AgentCore 런타임(Runtime)은 세션 간 완전한 격리를 보장합니다:

- **보안**: 각 세션이 격리된 리소스가 있는 자체 microVM에서 실행됨
- **프라이버시**: 서로 다른 사용자 세션 간 데이터 유출 없음
- **신뢰성**: 한 세션의 문제가 다른 세션에 영향을 미치지 않음
- **정리**: 세션 종료 후 완전한 메모리(Memory) 소독

### 페이로드(Payload) 유연성

`payload` 매개변수는 유연한 데이터 전달을 허용합니다:

```python
# 페이로드(Payload) 구조 예시
payload = {
    "prompt": "사용자 질문",
    "user_id": "1",
    "preferences": {...},
    "context_data": {...}
}
```

이를 통해 런타임(Runtime)에서 제공하는 세션 컨텍스트를 유지하면서 클라이언트와 에이전트(Agent) 간의 풍부하고 구조화된 통신이 가능합니다.

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configuration)

다음으로 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configuration)합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 키트를 구성(Configuration)합니다.

구성(Configuration) 단계에서 애플리케이션 코드를 기반으로 도커 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## Demonstrating Session Management and Context Handling

Now let's demonstrate the key session management features of AgentCore Runtime by testing different scenarios:

### Scenario 1: Session Continuity
We'll use the same session ID for multiple invocations to show how context is maintained.

### Scenario 2: Session Isolation
We'll use different session IDs to demonstrate complete isolation between sessions.

### Scenario 3: Context Information Access
We'll show how agents can access runtime context information.

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
import uuid
import json
from IPython.display import Markdown, display

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()
print(f"📋 Starting Session 1: {session_id}")
print(f"👤 User: Maira (ID: 1)")
print(f"❓ First question about weather\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather outside?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# Continue with the same session ID to demonstrate session continuity
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Follow-up question about math\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How much is 2X5?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# Continue with the same session ID - notice how the agent remembers the previous calculation
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Building on previous answer - demonstrates context continuity\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "and that plus 34?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# NEW SESSION - Demonstrate session isolation
# Create a completely new session ID to show that context is lost
new_session_id = uuid.uuid4()
print(f"🆕 Starting NEW Session 2: {new_session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Attempting to reference previous calculation - should fail due to session isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "And plus 10?",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# NEW SESSION AND USER - Demonstrate complete isolation
different_user_session = uuid.uuid4()
print(f"🆕 Starting Session 3: {different_user_session}")
print(f"👤 Different user: Mani (ID: 2)")
print(f"❓ Same question as first user - demonstrates user isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather?",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

### AgentCore 런타임(Runtime)에 컨텍스트 인식 에이전트(Agent) 시작

이제 도커 파일이 준비되었으니 컨텍스트 인식 에이전트(Agent)를 AgentCore 런타임(Runtime)에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore 런타임(Runtime)이 생성됩니다.

에이전트(Agent)는 AgentCore 런타임(Runtime)이 세션을 관리하고 에이전트(Agent)에 컨텍스트 정보를 제공하는 방법을 시연합니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!

You have successfully implemented and tested session management and context handling with Amazon Bedrock AgentCore Runtime! 

## What you've learned:

### Session Management Fundamentals
* **Session Continuity**: How the same session ID maintains context across multiple invocations
* **Session Isolation**: How different session IDs create completely isolated environments
* **Context Preservation**: How agents can maintain state and reference previous interactions
* **Security Boundaries**: How AgentCore Runtime ensures complete isolation between users

### Runtime Context Handling
* **Context Object Access**: How to access runtime information via the `context` parameter
* **Session Information**: How to retrieve and use session IDs in your agent logic
* **Payload Processing**: How to handle structured data passed through custom payloads
* **Runtime Metadata**: How agents can access execution environment information

### AgentCore Runtime Architecture
* **MicroVM Isolation**: Each session runs in its own isolated microVM
* **Resource Management**: Dedicated CPU, memory, and filesystem per session
* **Security Model**: Complete memory sanitization after session termination
* **Lifecycle Management**: Session states (active, idle, terminated) and timeouts

### Best Practices Implementation
* **Session ID Generation**: Creating unique identifiers for different conversations
* **Context Utilization**: Leveraging runtime context for enhanced agent behavior
* **State Management**: Understanding ephemeral vs persistent state
* **Error Handling**: Graceful handling of context loss and session boundaries